# Importing Libraries and Loading Datasets

To kick off the analysis, we start by importing the necessary libraries. The `pandas` library is our go-to tool for data manipulation, and we set some display options to ensure we can see all columns and a limited number of rows for better readability. Next, we load all the relevant datasets, including customer information, geolocation data, order details, product information, seller data, reviews, payments, and even a translation for product categories. These datasets will serve as the foundation for our analysis.

In [2]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

# Define dataset folder path (works for both Windows & Mac)
dataset_folder = os.path.join("dataset", "00_original_dataset")

# Load datasets using os.path.join()
customers = pd.read_csv(os.path.join(dataset_folder, "olist_customers_dataset.csv"))
geolocation = pd.read_csv(os.path.join(dataset_folder, "olist_geolocation_dataset.csv"))
order_items = pd.read_csv(os.path.join(dataset_folder, "olist_order_items_dataset.csv"))
orders = pd.read_csv(os.path.join(dataset_folder, "olist_orders_dataset.csv"))
products = pd.read_csv(os.path.join(dataset_folder, "olist_products_dataset.csv"))
sellers = pd.read_csv(os.path.join(dataset_folder, "olist_sellers_dataset.csv"))
order_reviews = pd.read_csv(os.path.join(dataset_folder, "olist_order_reviews_dataset.csv"))
order_payments = pd.read_csv(os.path.join(dataset_folder, "olist_order_payments_dataset.csv"))
category_translation = pd.read_csv(os.path.join(dataset_folder, "product_category_name_translation.csv"))

print("Datasets loaded successfully!")

Datasets loaded successfully!


# Filling Missing ZIP code

We will check if all the ZIP in seller and customer are already in geolocation, if not we will try to fill that with nearest long and lat area.

In [3]:
# Get unique zip codes
customer_zip_unique = set(customers["customer_zip_code_prefix"].unique())
seller_zip_unique = set(sellers["seller_zip_code_prefix"].unique())
geo_zip_unique = set(geolocation["geolocation_zip_code_prefix"].unique())

# Find missing zip codes 
missing_customer_zip = customer_zip_unique - geo_zip_unique
missing_seller_zip = seller_zip_unique - geo_zip_unique

# Print results
print(f"🔍 Missing Customer Zip Codes: {len(missing_customer_zip)} found")
print(missing_customer_zip if len(missing_customer_zip) < 20 else list(missing_customer_zip)[:20])

print(f"\n🔍 Missing Seller Zip Codes: {len(missing_seller_zip)} found")
print(missing_seller_zip if len(missing_seller_zip) < 20 else list(missing_seller_zip)[:20])

🔍 Missing Customer Zip Codes: 157 found
[28160, 56327, 75784, 29196, 71698, 68629, 29718, 70686, 67105, 73255, 71208, 12332, 70701, 70702, 72238, 72237, 7729, 72242, 72243, 64047]

🔍 Missing Seller Zip Codes: 7 found
{72580, 37708, 2285, 7412, 82040, 91901, 71551}


In [4]:
from geopy.distance import great_circle
# Keep only unique zip codes from geolocation
geo_unique = geolocation.drop_duplicates(subset="geolocation_zip_code_prefix")

# Function to find the nearest available zip code
def find_nearest_zip(missing_zip, geo_df):
    min_dist = float('inf')
    nearest_lat, nearest_lng = None, None

    for _, row in geo_df.iterrows():
        lat, lng = row["geolocation_lat"], row["geolocation_lng"]
        
        # Calculate distance using Haversine formula
        dist = great_circle((lat, lng), (0, 0)).km  # Approximate reference point
        
        if dist < min_dist:
            min_dist = dist
            nearest_lat, nearest_lng = lat, lng

    return nearest_lat, nearest_lng

# Find missing customer zip codes
missing_customer_zips = set(customers["customer_zip_code_prefix"].unique()) - set(geo_unique["geolocation_zip_code_prefix"].unique())

# Create a DataFrame for missing zip codes
missing_customer_geo = pd.DataFrame(
    [(zip_code, *find_nearest_zip(zip_code, geo_unique)) for zip_code in missing_customer_zips],
    columns=["geolocation_zip_code_prefix", "geolocation_lat", "geolocation_lng"]
)

# Find missing seller zip codes
missing_seller_zips = set(sellers["seller_zip_code_prefix"].unique()) - set(geo_unique["geolocation_zip_code_prefix"].unique())

# Create a DataFrame for missing seller zip codes
missing_seller_geo = pd.DataFrame(
    [(zip_code, *find_nearest_zip(zip_code, geo_unique)) for zip_code in missing_seller_zips],
    columns=["geolocation_zip_code_prefix", "geolocation_lat", "geolocation_lng"]
)

# Concatenate with the original geolocation dataset
geolocation_fixed = pd.concat([geolocation, missing_customer_geo, missing_seller_geo], ignore_index=True)



# Create the folder if it doesn't exist
folder_path = "dataset/00_geoloc_data"
os.makedirs(folder_path, exist_ok=True)

# Define file paths
geoloc_dataset_path = os.path.join(folder_path, "Fixed_Geolocation.csv")

# Save DataFrames to CSV
geolocation_fixed.to_csv(geoloc_dataset_path, index=False)

print("✅ Missing zip codes assigned and saved successfully!")

✅ Missing zip codes assigned and saved successfully!


In [5]:
geo = pd.read_csv("dataset/00_geoloc_data/Fixed_Geolocation.csv")

In [6]:
geo[geo['geolocation_zip_code_prefix']==38710]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
1000261,38710,28.008978,-15.536867,NaN,NaN


In [7]:
geolocation_fixed = pd.read_csv("dataset/00_geoloc_data/Fixed_Geolocation.csv")

# Load city & state mappings from customers and sellers
customer_zip_mapping = customers.groupby("customer_zip_code_prefix").agg({"customer_city": "first", "customer_state": "first"})
seller_zip_mapping = sellers.groupby("seller_zip_code_prefix").agg({"seller_city": "first", "seller_state": "first"})

# Rename columns for merging
customer_zip_mapping = customer_zip_mapping.rename(columns={"customer_city": "geolocation_city", "customer_state": "geolocation_state"})
seller_zip_mapping = seller_zip_mapping.rename(columns={"seller_city": "geolocation_city", "seller_state": "geolocation_state"})

# Combine both mappings (customers & sellers)
zip_mapping = pd.concat([customer_zip_mapping, seller_zip_mapping]).reset_index().drop_duplicates(subset="index")
zip_mapping = zip_mapping.rename(columns={"index": "geolocation_zip_code_prefix"})

# Merge the mapping into the geolocation dataset
geolocation_filled = geolocation_fixed.merge(zip_mapping, on="geolocation_zip_code_prefix", how="left", suffixes=("", "_new"))

# Fill missing values from the mapping
geolocation_filled["geolocation_city"].fillna(geolocation_filled["geolocation_city_new"], inplace=True)
geolocation_filled["geolocation_state"].fillna(geolocation_filled["geolocation_state_new"], inplace=True)

# Drop helper columns
geolocation_filled = geolocation_filled.drop(columns=["geolocation_city_new", "geolocation_state_new"])


# Define dataset folder path (cross-platform)
folder_path = os.path.join("dataset", "00_geoloc_data")

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define file paths using os.path.join()
geoloc_dataset_path = os.path.join(folder_path, "Final_Geolocation.csv")

# Save DataFrame to CSV
geolocation_filled.to_csv(geoloc_dataset_path, index=False)

print("✅ Missing city & state values have been filled using customers & sellers data!")


/var/folders/dt/mp18g7bj3fs8hm27kth1hcvw0000gn/T/ipykernel_23298/2773666176.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  geolocation_filled["geolocation_city"].fillna(geolocation_filled["geolocation_city_new"], inplace=True)
/var/folders/dt/mp18g7bj3fs8hm27kth1hcvw0000gn/T/ipykernel_23298/2773666176.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because th

✅ Missing city & state values have been filled using customers & sellers data!


In [8]:
geolocation[geolocation['geolocation_zip_code_prefix']==38710]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


In [9]:
geolocation = geolocation_filled

# Grouping Geolocation Data

The geolocation dataset contains detailed information about zip codes, cities, states, and their corresponding latitude and longitude. To simplify this data, we group it by the `geolocation_zip_code_prefix` column. For each unique zip code prefix, we calculate the average latitude and longitude and take the first occurrence of the city and state. This grouped data will later help us merge geolocation information with other datasets based on zip codes.

In [10]:
geo_grouped = geolocation.groupby("geolocation_zip_code_prefix").agg({
    "geolocation_lat": "mean",
    "geolocation_lng": "mean",
    "geolocation_city": "first",
    "geolocation_state": "first"
}).reset_index()

# Grouping Order Reviews and Payments

In this step, we focus on simplifying the order reviews and payments data. For each order, there might be multiple reviews or payments, so we group the data by `order_id` and calculate the **median** `review_score` and `payment_value` for each order. This reduces the complexity of the data and ensures that each order is represented by a single review score and payment value, making it easier to analyze. By using the median, we mitigate the impact of any outliers in the data.

In [11]:
order_reviews = (
    order_reviews.groupby('order_id')['review_score']
    .median()
    .apply(np.floor)
    .reset_index()
)
order_payments = order_payments.groupby('order_id').agg({
    # 'payment_value': 'median',  
    'payment_type': lambda x: x.mode()[0]  # Get the most frequent (mode) payment type
}).reset_index()

In [12]:
order_reviews['review_score'].unique()

array([5., 4., 1., 3., 2.])

# Creating Business, Seller, and Buyer DataFrames

Here, we create three main DataFrames to analyze the data from different perspectives: business, seller, and buyer. 

- **Business Side**: This DataFrame combines order items, orders, reviews, customer details, product information, category translations, and geolocation data. We also calculate two new columns: the average number of items per order and the net profit (which is 18% of the product price). This gives us a comprehensive view of the business operations.

- **Seller Side**: This DataFrame merges order items, orders, reviews, product details, category translations, seller information, and geolocation data. We also calculate the time it takes for sellers to deliver items to the carrier and the net profit. This helps us understand the seller’s performance and logistics.

- **Buyer Side**: This DataFrame focuses on the buyer’s perspective by combining order items, orders, reviews, customer details, product information, category translations, payment data, and geolocation. It includes key details like payment values and review scores, giving us insights into the buyer’s experience.

Finally, we select only the relevant columns for each DataFrame to keep the data clean and focused.

In [13]:
# Business Side
business_df = order_items.merge(orders, on="order_id", how="left")
business_df = business_df.merge(order_reviews, on="order_id", how="left")
business_df = business_df.merge(customers, on="customer_id", how="left")
business_df = business_df.merge(products, on="product_id", how="left")
business_df = business_df.merge(category_translation, on='product_category_name', how='left')
business_df = business_df.merge(
    geo_grouped,
    left_on=['customer_zip_code_prefix'],
    right_on=['geolocation_zip_code_prefix'],
    how='left'
)

# Calculate new columns for Business Side
business_df["avg_items_per_order"] = business_df.groupby("order_id")["order_item_id"].transform("count")
business_df["net_profit"] = business_df["price"] * 0.18

# Seller Side
seller_df = order_items.merge(orders, on="order_id", how="left")
seller_df = seller_df.merge(order_reviews, on="order_id", how="left")
seller_df = seller_df.merge(products, on="product_id", how="left")
seller_df = seller_df.merge(category_translation, on='product_category_name', how='left')
seller_df = seller_df.merge(sellers, on="seller_id", how="left")
seller_df = seller_df.merge(
    geo_grouped,
    left_on=['seller_zip_code_prefix'],
    right_on=['geolocation_zip_code_prefix'],
    how='left'
)

# Calculate new columns for Seller Side
seller_df["time_deliver_to_carrier"] = pd.to_datetime(seller_df["order_delivered_carrier_date"]) - pd.to_datetime(seller_df["order_approved_at"])
seller_df["net_profit"] = seller_df["price"] * 0.18

# Buyer Side
buyer_df = order_items.merge(orders, on="order_id", how="left")
buyer_df = buyer_df.merge(order_reviews, on="order_id", how="left")
buyer_df = buyer_df.merge(customers, on="customer_id", how="left")
buyer_df = buyer_df.merge(products, on="product_id", how="left")
buyer_df = buyer_df.merge(category_translation, on='product_category_name', how='left')
buyer_df = buyer_df.merge(order_payments, on="order_id", how="left")
buyer_df = buyer_df.merge(
    geo_grouped,
    left_on=['customer_zip_code_prefix'],
    right_on=['geolocation_zip_code_prefix'],
    how='left'
)

# Final Columns
business_side = [
    'customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_state',
    'geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state',
    'product_id', 'product_category_name_english',
    'price',
    'order_id', 'order_item_id', 'order_purchase_timestamp',
    'avg_items_per_order', 'net_profit','order_status',
]

seller_side = [
    'seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state',
    'geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state',
    'product_id', 'product_category_name_english',
    'price', 
    'order_id', 'order_item_id', 'order_purchase_timestamp', 'order_approved_at', 
    'shipping_limit_date',
    'order_status',
    'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date',
    'review_score',
    'time_deliver_to_carrier', 'net_profit'
]

buyer_side = [
    'customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_state',
    'geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state',
    'product_id', 'product_category_name_english',
    'price',
    'order_id', 'order_item_id',
    'order_purchase_timestamp',
    # 'payment_value',
    'payment_type',
    'order_status',
    'review_score'
]

business_df = business_df[business_side]
seller_df = seller_df[seller_side]
buyer_df = buyer_df[buyer_side]

# Changing Data Types

To ensure the data is in the right format for analysis, we convert certain columns to appropriate data types:

- **Business Side**: The `order_purchase_timestamp` is converted to a datetime format, and zip code prefixes are treated as strings to avoid any numerical misinterpretation.

- **Seller Side**: Similar to the business side, zip code prefixes are converted to strings. Additionally, various date-related columns (like `order_purchase_timestamp`, `order_approved_at`, etc.) are converted to datetime format. The `time_deliver_to_carrier` column, which represents the time difference between order approval and delivery to the carrier, is converted to a timedelta for easier time-based calculations.

- **Buyer Side**: Zip code prefixes are converted to strings to maintain consistency across all DataFrames.

These conversions are crucial for accurate data analysis and manipulation.

In [14]:
# Change Dtype
# Business Side
business_df['order_purchase_timestamp'] = pd.to_datetime(business_df['order_purchase_timestamp'])
business_df['customer_zip_code_prefix'] = business_df['customer_zip_code_prefix'].astype(str)
business_df['geolocation_zip_code_prefix'] = business_df['geolocation_zip_code_prefix'].astype(str)

# Seller Side
seller_df['seller_zip_code_prefix'] = seller_df['seller_zip_code_prefix'].astype(str)
seller_df['geolocation_zip_code_prefix'] = seller_df['geolocation_zip_code_prefix'].astype(str)
datetime_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'shipping_limit_date',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]


for col in datetime_columns:
    seller_df[col] = pd.to_datetime(seller_df[col])

seller_df['time_deliver_to_carrier'] = pd.to_timedelta(seller_df['time_deliver_to_carrier'])

#Buyer Side
buyer_df['order_purchase_timestamp'] = pd.to_datetime(buyer_df['order_purchase_timestamp'])
buyer_df['customer_zip_code_prefix'] = buyer_df['customer_zip_code_prefix'].astype(str)
buyer_df['geolocation_zip_code_prefix'] = buyer_df['geolocation_zip_code_prefix'].astype(str)

# Inspecting Business DataFrame

Now that we’ve prepared the `business_df` DataFrame, we take a closer look at its structure using the `info()` method. This gives us a summary of the DataFrame, including the number of non-null entries, data types, and memory usage for each column. Inspecting the DataFrame not only helps us understand its structure but also ensures that the data types (dtypes) are correct. For example, we confirm that datetime columns are properly formatted and that zip codes are treated as strings. This step is essential to avoid errors in later analysis.

In [15]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_id                    112650 non-null  object        
 1   customer_unique_id             112650 non-null  object        
 2   customer_zip_code_prefix       112650 non-null  object        
 3   customer_state                 112650 non-null  object        
 4   geolocation_zip_code_prefix    112650 non-null  object        
 5   geolocation_lat                112650 non-null  float64       
 6   geolocation_lng                112650 non-null  float64       
 7   geolocation_city               112650 non-null  object        
 8   geolocation_state              112650 non-null  object        
 9   product_id                     112650 non-null  object        
 10  product_category_name_english  111023 non-null  object        
 11  

# Inspecting Seller DataFrame

Similarly, we inspect the `seller_df` DataFrame using the `info()` method. This provides a detailed overview of the seller-side data, including the number of non-null entries, data types, and memory usage. By checking the dtypes, we ensure that all columns are in the correct format, such as datetime columns for order dates and timedelta for delivery times. This step helps us confirm that the seller-related data is clean and ready for analysis.

In [16]:
seller_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 24 columns):
 #   Column                         Non-Null Count   Dtype          
---  ------                         --------------   -----          
 0   seller_id                      112650 non-null  object         
 1   seller_zip_code_prefix         112650 non-null  object         
 2   seller_city                    112650 non-null  object         
 3   seller_state                   112650 non-null  object         
 4   geolocation_zip_code_prefix    112650 non-null  object         
 5   geolocation_lat                112650 non-null  float64        
 6   geolocation_lng                112650 non-null  float64        
 7   geolocation_city               112650 non-null  object         
 8   geolocation_state              112650 non-null  object         
 9   product_id                     112650 non-null  object         
 10  product_category_name_english  111023 non-null  object  

# Inspecting Buyer DataFrame

Next, we inspect the `buyer_df` DataFrame using the `info()` method. This gives us a summary of the buyer-side data, including the number of non-null entries, data types, and memory usage. By checking the dtypes, we ensure that all columns are in the correct format, such as zip codes being treated as strings and payment values as floats. This step helps us confirm that the buyer-related data is clean and ready for analysis, ensuring that the buyer’s perspective is accurately represented.

In [17]:
buyer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_id                    112650 non-null  object        
 1   customer_unique_id             112650 non-null  object        
 2   customer_zip_code_prefix       112650 non-null  object        
 3   customer_state                 112650 non-null  object        
 4   geolocation_zip_code_prefix    112650 non-null  object        
 5   geolocation_lat                112650 non-null  float64       
 6   geolocation_lng                112650 non-null  float64       
 7   geolocation_city               112650 non-null  object        
 8   geolocation_state              112650 non-null  object        
 9   product_id                     112650 non-null  object        
 10  product_category_name_english  111023 non-null  object        
 11  

# Summing Prices Across DataFrames

To ensure consistency across all DataFrames, we calculate the total sum of the `price` column in each DataFrame: `business_df`, `seller_df`, and `buyer_df`. The total sum should be the same across all three DataFrames because they all reference the same underlying order items. This check confirms that the data merging and transformations haven’t introduced any discrepancies. In this case, the total sum matches across all DataFrames, which is a good sign that the data is consistent and reliable.

In [18]:
business_df.price.sum()

13591643.700000003

In [19]:
seller_df.price.sum()

13591643.700000003

In [20]:
buyer_df.price.sum()

13591643.700000003

# Checking for Duplicates

Before proceeding further, it’s important to ensure that our data is clean and free of duplicates. We check for duplicate rows in each DataFrame using the `duplicated()` method. The results show that there are **no duplicates** in any of the DataFrames (`business_df`, `seller_df`, or `buyer_df`). This confirms that the data is clean and ready for analysis, with no repeated rows that could skew our results.

In [21]:
business_df.duplicated().sum()

0

In [22]:
seller_df.duplicated().sum()

0

In [23]:
buyer_df.duplicated().sum()

0

# Saving DataFrames to CSV

Once all the data processing is complete, we save the three DataFrames (`business_df`, `seller_df`, and `buyer_df`) to CSV files. These files are named `business_side.csv`, `seller_side.csv`, and `buyer_side.csv`, respectively. Saving the data to CSV files allows us to easily share or use the processed data for further analysis or reporting. This step ensures that the cleaned and transformed data is preserved for future use.

In [24]:
import os

# Define dataset folder path (cross-platform)
folder_path = os.path.join("dataset", "01_three_side_dataset")

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define file paths using os.path.join()
business_path = os.path.join(folder_path, "business_side.csv")
seller_path = os.path.join(folder_path, "seller_side.csv")
buyer_path = os.path.join(folder_path, "buyer_side.csv")

# Save DataFrames to CSV
business_df.to_csv(business_path, index=False)
seller_df.to_csv(seller_path, index=False)
buyer_df.to_csv(buyer_path, index=False)

print("CSV files saved successfully!")


CSV files saved successfully!
